## Installation de packages et importation

In [ ]:
!pip install -r requirements.txt

In [ ]:
pip install yahoo_fin

In [ ]:
import html5lib 
import requests_html
import datetime
import feedparser
import io
import json
import pandas
import requests
import requests_html
import html5lib 
import pandas as pd
import numpy as np
from yahoo_fin import options
from yahoo_fin.stock_info import get_data
from datetime import datetime

## Téléchargement et premiers traitements sur les données d'options

In [ ]:

tickers= ['aapl', 'amzn', 'tsla', 'googl', 'meta', 'msft', 'sony', 'baba']
def get_options_price(ticker):
    """ 
    Enregistre un fichier csv contenant les données d'options de sous jacent le ticker spécifié
    ticker: str
    """
    ListOptions = []
    dates = options.get_expiration_dates(ticker)
    for date in dates :
        data= options.get_options_chain(date = date, ticker=ticker)
        dataDict = {'Call' : data["calls"], 'Put' : data["puts"]}
        for typ, data in dataDict.items():
            data["Type"] = typ
            data["Maturity"] = date
            ListOptions.append(data)
    list_name = 'ListAllOptions'+ticker+'.csv'
    ListAllOptions = pd.concat(ListOptions)
    ListAllOptions.to_csv(list_name, index=False)

In [ ]:
for ticker in tickers:
    get_options_price(ticker)

In [ ]:
for ticker in tickers:
    name = "ListAllOptions"+ticker+".csv"
    ListAllOptions = pd.read_csv(name)
    ListAllOptions['Volume'] = ListAllOptions['Volume'].replace('-',0).astype('float')
    ListAllOptions['Strike'] = ListAllOptions['Strike'].astype('float')
    ListAllOptions['Last Price'] = ListAllOptions['Last Price'].astype('float')
    ListAllOptions['Bid'] = ListAllOptions['Bid'].astype('float')
    ListAllOptions['Ask'] = ListAllOptions['Ask'].astype('float')
    ListAllOptions['Maturity'] = pd.to_datetime(ListAllOptions['Maturity'], format='%B %d, %Y')
    ListAllOptions['index'] = range(0,len(ListAllOptions))

    # Keep one of the call or the put regarding which one have more transactions
    index_to_keep = []

    for date in ListAllOptions['Maturity']:
        date_data = ListAllOptions[ListAllOptions['Maturity']==date]
        for strike in date_data['Strike']:
            ds_data = date_data[date_data['Strike']== strike]
            volume = list(ds_data['Volume'])
            index = list(ds_data['index'])
            max_volume = max(volume)
            index_to_keep.append(index[volume.index(max_volume)])
    index_to_keep = np.unique(index_to_keep)

    # Retirer les options non liquides : Nous considérons arbitrairement qu'une 
    # option n'est pas liquide lorsque son volume de transactions est inférieure à la
    # médiane des transactions (qui est généralement de l'ordre de la dizaine)
    volume = ListAllOptions['Volume']
    median_volume = np.median(volume)
    ListAllOptions = ListAllOptions[ListAllOptions['Volume']>=median_volume]

    list_name = 'cleaned_ListAllOptions'+ticker+'.csv'
    ListAllOptions.to_csv(list_name, index=False)

## Téléchargement d'historisque de cours d'actions pour le calcul de la volatilité historique

In [ ]:
def get_stock_prices(ticker) :
    """ 
    Enregistre un fichier csv contenant l'historique des cours du ticker spécifié, avec comme date de départ le 01/01/2021 (après le covid)
    ticker: str
    """
    data = get_data(ticker, start_date = '01/01/2021')
    if ticker == 'aapl':
        name = 'APPLE'
    elif ticker=="amzn":
        name="AMAZON"
    elif ticker=="baba":
        name="ALI BABA"
    elif ticker=="googl":
        name="GOOGLE"
    elif ticker=="meta":
        name="META"
    elif ticker=="msft":
        name="MICROSOFT"
    elif ticker=="sony":
        name="SONY"
    elif ticker=="tsla":
        name="TESLA"
    data = data[['close']]
    data_name = f'StockPrices{name}.csv'
    data.to_csv(f'C:/Users/HP/Documents/GitHub/asset-pricing/src/data/{data_name}')

In [ ]:
for ticker in tickers:
    get_stock_prices(ticker)